## Flowers Image Classification using a Linear Model

In this notebook, we will use a creative-commons licensed flower photo dataset
containing **3,670 images** across **5 categories**:

- daisy
- dandelion
- roses
- sunflowers
- tulips

Rather than relying on cloud storage, we will load this dataset locally using
`tensorflow_datasets`, which provides a standardized and reproducible way to
access common machine learning datasets.

The dataset will be:
- Downloaded once (if not already cached)
- Stored locally in the `data/tfds/` directory
- Split into training (80%) and validation (20%) subsets

This setup allows the notebook to run fully offline after the initial download
and ensures consistent results across student machines.

### 📁 Visual Outputs

To ensure this notebook runs reliably on Windows, all figures are **saved to disk**
instead of being displayed inline.

➡️ After running the notebook, open the `outputs/` folder to view:
- example images
- training curves
- predictions
- learned weights

If you are on macOS/Linux, you may replace savefig() with plt.show() to see the images without saving them.

In [1]:
# ========================================================
# Step 0: Imports and environment setup
# ========================================================

from pathlib import Path
import platform
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from PIL import Image
import csv

tf.random.set_seed(42)

# -------------------------------
# Windows TensorFlow stability
# -------------------------------
if platform.system() == "Windows":
    # Disable GPU to prevent native crashes
    try:
        tf.config.set_visible_devices([], 'GPU')
    except:
        pass

    # Limit threading to avoid MKL / OpenMP conflicts
    tf.config.threading.set_intra_op_parallelism_threads(1)
    tf.config.threading.set_inter_op_parallelism_threads(1)

# -------------------------------
# Output directories
# -------------------------------
DATA_DIR = Path("data")
OUT_DIR = Path("outputs")

DATA_DIR.mkdir(exist_ok=True)
OUT_DIR.mkdir(exist_ok=True)

print(f"Dataset directory: {DATA_DIR.resolve()}")
print(f"Output directory:  {OUT_DIR.resolve()}")

# ========================================================
# Step 1: Load the TF Flowers dataset
# ========================================================

(ds_train, ds_val), ds_info = tfds.load(
    "tf_flowers",
    split=["train[:80%]", "train[80%:]"],
    as_supervised=True,
    with_info=True,
    data_dir=DATA_DIR / "tfds"
)

CLASS_NAMES = ds_info.features["label"].names
NUM_CLASSES = ds_info.features["label"].num_classes

print(f"Number of classes: {NUM_CLASSES}")
print("Class names:", CLASS_NAMES)
print(f"Training samples: {ds_info.splits['train'].num_examples}")
print(f"Validation samples: {int(ds_info.splits['train'].num_examples * 0.2)}")


Dataset directory: C:\Users\Jason Eckert\Documents\cv\02_ml_models\data
Output directory:  C:\Users\Jason Eckert\Documents\cv\02_ml_models\outputs
Number of classes: 5
Class names: ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']
Training samples: 3670
Validation samples: 734


In [2]:
# ========================================================
# Step 2: Display a few images safely (Windows)
# ========================================================
print("\n--- Saving one example image per class ---")

for idx, class_name in enumerate(CLASS_NAMES):
    class_ds = ds_train.filter(lambda img, lbl: lbl == idx)
    for image, label in class_ds.take(1):
        img_uint8 = image.numpy().astype("uint8")
        img_pil = Image.fromarray(img_uint8)
        img_pil.save(OUT_DIR / f"class_example_{class_name}.png")
        print(f"Saved example for class '{class_name}'")



--- Saving one example image per class ---
Saved example for class 'dandelion'
Saved example for class 'daisy'
Saved example for class 'tulips'
Saved example for class 'sunflowers'
Saved example for class 'roses'


## A simple rule-based model

Let's get the average color of RGB values in the different
types of flowers and then classify an unknown image as
belonging to closest centroid.

In [3]:
# ========================================================
# Step 3: Compute average RGB per image (rule-based centroid)
# ========================================================
print("\n--- Per-image average RGB (first 3 images) ---")

for image, label in ds_train.take(3):
    avg_color = tf.reduce_mean(tf.cast(image, tf.float32), axis=[0, 1])
    print(CLASS_NAMES[int(label)], avg_color.numpy())



--- Per-image average RGB (first 3 images) ---
tulips [156.44687   72.975204  31.295033]
sunflowers [82.408905 57.083298 19.67777 ]
sunflowers [125.994156 142.72287   94.961655]


In [4]:
# ========================================================
# Step 4a: Define Centroid and CentroidRule classes
# ========================================================

class Centroid:
    def __init__(self, label):
        self.label = label
        self.sum_so_far = tf.constant(0., dtype=tf.float32)
        self.count_so_far = 0

    def update(self, value):
        self.sum_so_far += value
        self.count_so_far += 1
        if self.count_so_far % 100 == 0:
            print(self.label, self.count_so_far)

    def centroid(self):
        return self.sum_so_far / self.count_so_far

    def __str__(self):
        return f'{self.label} {self.centroid().numpy()}'


class CentroidRule:
    def __init__(self):
        self.centroids = {f: Centroid(f) for f in CLASS_NAMES}

    def fit(self, dataset):
        for img, label in dataset:
            label_name = CLASS_NAMES[int(label)]
            avg = tf.reduce_mean(tf.cast(img, tf.float32), axis=[0, 1])
            self.centroids[label_name].update(avg)

    def predict(self, img):
        avg = tf.reduce_mean(tf.cast(img, tf.float32), axis=[0, 1])
        best_label = ""
        best_diff = float("inf")

        for key, val in self.centroids.items():
            diff = tf.reduce_sum(tf.abs(avg - val.centroid()))
            if diff < best_diff:
                best_diff = diff
                best_label = key

        return best_label

    def evaluate(self, dataset):
        correct, total = 0, 0
        for img, label in dataset:
            if self.predict(img) == CLASS_NAMES[int(label)]:
                correct += 1
            total += 1
        return correct / total


In [5]:
# ========================================================
# Step 4b: Fit centroid classifier on small subsets (fast demo)
# ========================================================
train_subset = ds_train.take(500)
eval_subset = ds_val.take(50)

rule = CentroidRule()
rule.fit(train_subset)

print("\nCentroid for daisy:", rule.centroids["daisy"])
print("Centroid for roses:", rule.centroids["roses"])


dandelion 100
tulips 100

Centroid for daisy: daisy [103.03394  110.325645  86.47161 ]
Centroid for roses: roses [127.029045  96.59135   87.69432 ]


In [6]:
# ========================================================
# Step 4c: Evaluate classifier on small validation subset
# ========================================================

accuracy = rule.evaluate(eval_subset)
print("\nCentroid-rule accuracy:", accuracy)



Centroid-rule accuracy: 0.36


In [7]:
# ========================================================
# Step 5a: Using the model to predict one image
# ========================================================

from PIL import Image

# Directory to save example images
RULE_OUTPUT_DIR = OUT_DIR / "rule_based_examples"
RULE_OUTPUT_DIR.mkdir(exist_ok=True)

print("\n--- Rule-Based Predictions (Validation set) ---")

# Take one example
for idx, (img, label) in enumerate(ds_val.take(1)):
    predicted_label = rule.predict(img)
    true_label = CLASS_NAMES[int(label)]
    print(f"Example {idx+1}")
    print("  True label:     ", true_label)
    print("  Predicted label:", predicted_label)

    # Save the image with predicted label in filename
    img_pil = Image.fromarray(img.numpy().astype("uint8"))
    img_pil.save(RULE_OUTPUT_DIR / f"example_{idx+1}_{true_label}_pred_{predicted_label}.png")



--- Rule-Based Predictions (Validation set) ---
Example 1
  True label:      roses
  Predicted label: dandelion


In [8]:
# ========================================================
# Step 5b: Using the model to predict a small batch
# ========================================================

# Take a small batch of 5 examples
for idx, (img, label) in enumerate(ds_val.take(5)):
    predicted_label = rule.predict(img)
    true_label = CLASS_NAMES[int(label)]
    print(f"Batch example {idx+1}")
    print("  True label:     ", true_label)
    print("  Predicted label:", predicted_label)

    # Save image
    img_pil = Image.fromarray(img.numpy().astype("uint8"))
    img_pil.save(RULE_OUTPUT_DIR / f"batch_{idx+1}_{true_label}_pred_{predicted_label}.png")

print(f"\nSaved rule-based example images to {RULE_OUTPUT_DIR.resolve()}")

Batch example 1
  True label:      roses
  Predicted label: dandelion
Batch example 2
  True label:      tulips
  Predicted label: dandelion
Batch example 3
  True label:      tulips
  Predicted label: dandelion
Batch example 4
  True label:      tulips
  Predicted label: roses
Batch example 5
  True label:      daisy
  Predicted label: dandelion

Saved rule-based example images to C:\Users\Jason Eckert\Documents\cv\02_ml_models\outputs\rule_based_examples


## A linear model

In [9]:
# ========================================================
# Step 6: Prepare datasets for linear model
# ========================================================

BATCH_SIZE = 10
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 224, 224, 3

def preprocess(img, label):
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    return img, label

train_dataset = ds_train.map(preprocess).batch(BATCH_SIZE)
eval_dataset = ds_val.map(preprocess).batch(BATCH_SIZE)


In [10]:
# ========================================================
# Step 7: Define linear model
# ========================================================

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 150528)            0         
                                                                 
 dense (Dense)               (None, 5)                 752645    
                                                                 
Total params: 752,645
Trainable params: 752,645
Non-trainable params: 0
_________________________________________________________________


In [11]:
# ========================================================
# Step 8: Train the linear model
# ========================================================

history = model.fit(
    train_dataset,
    validation_data=eval_dataset,
    epochs=10
)


Epoch 1/10
294/294 [==============================] - 4s 11ms/step - loss: 4893.7021 - accuracy: 0.3331 - val_loss: 2577.3735 - val_accuracy: 0.4046
Epoch 2/10
294/294 [==============================] - 2s 6ms/step - loss: 3675.8147 - accuracy: 0.4074 - val_loss: 6270.2393 - val_accuracy: 0.3815
Epoch 3/10
294/294 [==============================] - 2s 6ms/step - loss: 3215.9446 - accuracy: 0.4493 - val_loss: 5716.2520 - val_accuracy: 0.4155
Epoch 4/10
294/294 [==============================] - 2s 7ms/step - loss: 3752.8423 - accuracy: 0.4636 - val_loss: 6338.2432 - val_accuracy: 0.4237
Epoch 5/10
294/294 [==============================] - 2s 6ms/step - loss: 3619.4319 - accuracy: 0.5000 - val_loss: 5228.4443 - val_accuracy: 0.3733
Epoch 6/10
294/294 [==============================] - 2s 6ms/step - loss: 3054.1499 - accuracy: 0.5303 - val_loss: 5253.6763 - val_accuracy: 0.3992
Epoch 7/10
294/294 [==============================] - 2s 6ms/step - loss: 3614.0764 - accuracy: 0.5204 - val_lo

In [12]:
# ========================================================
# Step 9: Save training curves as CSV (open/plot in Excel after)
# ========================================================
import csv

# CSV path
csv_path = OUT_DIR / "training_metrics.csv"

# Number of epochs
num_epochs = len(history.history["loss"])

# Create CSV in "Excel-friendly" layout
with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    # Header row: one column per metric
    writer.writerow(["Epoch", "Loss", "Validation Loss", "Accuracy", "Validation Accuracy"])

    # Write row per epoch
    for i in range(num_epochs):
        writer.writerow([
            i + 1,  # Epoch number (1-based)
            history.history["loss"][i],
            history.history["val_loss"][i],
            history.history["accuracy"][i],
            history.history["val_accuracy"][i],
        ])

print(f"Saved Excel-friendly training metrics to {csv_path}")


Saved Excel-friendly training metrics to outputs\training_metrics.csv


In Excel:
1. Open outputs/training_metrics.csv.
2. Select all rows including the header.
3. Insert → Charts → Line chart.

For Loss plot:
- y-axis = Loss / Validation Loss
- x-axis = Epoch

For Accuracy plot:
- y-axis = Accuracy / Validation Accuracy
- x-axis = Epoch

In [13]:
# ========================================================
# Step 10: Save predictions as image grid (PIL)
# ========================================================

from PIL import ImageDraw, ImageFont

def save_predictions_pil(dataset, num_images=15):
    # Take first num_images from dataset
    images, labels = next(iter(dataset.unbatch().batch(num_images)))
    preds = model.predict(images, verbose=0)

    grid_w, grid_h = 5, 3  # 5 columns, 3 rows
    img_h, img_w = images.shape[1], images.shape[2]

    canvas = Image.new("RGB", (grid_w * img_w, grid_h * img_h), "white")
    draw = ImageDraw.Draw(canvas)

    # Try to load a default font, fallback if not available
    try:
        font = ImageFont.truetype("arial.ttf", 18)
    except:
        font = ImageFont.load_default()

    for idx in range(num_images):
        img = Image.fromarray(images[idx].numpy().astype("uint8"))
        row, col = idx // grid_w, idx % grid_w
        x0, y0 = col * img_w, row * img_h

        # Paste the image into the canvas
        canvas.paste(img, (x0, y0))

        # Prediction info
        pred_probs = preds[idx]
        pred_idx = np.argmax(pred_probs)
        pred_label = CLASS_NAMES[pred_idx]
        true_label = CLASS_NAMES[int(labels[idx])]
        confidence = pred_probs[pred_idx]

        text = f"{true_label} → {pred_label} ({confidence:.2f})"

        # Measure text size (future-proof)
        try:
            bbox = draw.textbbox((0, 0), text, font=font)
            text_w = bbox[2] - bbox[0]
            text_h = bbox[3] - bbox[1]
        except AttributeError:
            text_w, text_h = draw.textsize(text, font=font)

        padding = 4
        box_coords = [
            x0,
            y0,
            x0 + text_w + 2 * padding,
            y0 + text_h + 2 * padding,
        ]

        # Draw background rectangle and text
        draw.rectangle(box_coords, fill=(0, 0, 0))
        draw.text(
            (x0 + padding, y0 + padding),
            text,
            fill=(255, 255, 255),
            font=font
        )

    # Save final canvas
    canvas.save(OUT_DIR / "linear_model_predictions.png")
    print("Saved prediction grid with labels and confidence to outputs/linear_model_predictions.png")

# Run it
save_predictions_pil(eval_dataset)


Saved prediction grid with labels and confidence to outputs/linear_model_predictions.png


In [14]:
# ========================================================
# Step 11: Visualize trained linear weights (PIL)
# ========================================================
def save_trained_weights_pil(model):
    WEIGHT_TYPE = 0
    LAYER = 1

    for flower in range(NUM_CLASSES):
        weights = model.layers[LAYER].get_weights()[WEIGHT_TYPE][:, flower]
        weights = weights.reshape(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

        # Downsample for clarity and stability
        weights = tf.image.resize(weights, [56, 56]).numpy()

        min_wt, max_wt = weights.min(), weights.max()
        weights = (weights - min_wt) / (max_wt - min_wt + 1e-8)

        img = Image.fromarray((weights * 255).astype("uint8"))
        img.save(OUT_DIR / f"weights_{CLASS_NAMES[flower]}.png")

save_trained_weights_pil(model)
print("Saved trained weight visualizations.")


Saved trained weight visualizations.


## Softmax Diagram


In [15]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import tensorflow as tf

# Example data
inx = [
    [0, 0.09, 0.06, 0.85, 0],
    [0.1, 0.1, 0.7, 0.1, 0.1],
    [0, 0.2, 0.4, 0.2, 0],
    [0.1, 0.1, 0.4, 0.5, 0.1],
    [0.2, 0.2, 0.8, 0.2, 0.2],
]

labels = ['A', 'B', 'C', 'D', 'E']

# Safe logit function
def logit(x):
    # Clip x to avoid divide-by-zero or log(0)
    x = tf.clip_by_value(x, 1e-7, 1 - 1e-7)
    return -tf.math.log(1. / x - 1.)

bar_width = 30
gap = 20
num_examples = len(inx)
num_categories = len(labels)
height_scale = 300  # pixels per 1.0 probability

# Canvas size
canvas_w = num_categories * 2 * bar_width + (num_categories + 1) * gap
canvas_h = num_examples * (height_scale + 50)
canvas = Image.new("RGB", (canvas_w, canvas_h), "white")
draw = ImageDraw.Draw(canvas)

# Font
try:
    font = ImageFont.truetype("arial.ttf", 16)
except:
    font = ImageFont.load_default()

for i, x in enumerate(inx):
    prob = np.array(x) / np.sum(x)
    logits = logit(prob)
    softmax = tf.nn.softmax(logits).numpy()

    for j, (p, s) in enumerate(zip(prob, softmax)):
        x0 = gap + j * (2 * bar_width + gap)
        y0_prob = i * (height_scale + 50) + height_scale - int(p * height_scale)
        y0_soft = i * (height_scale + 50) + height_scale - int(s * height_scale)
        y1 = i * (height_scale + 50) + height_scale

        # Draw original prob (blue)
        draw.rectangle([x0, y0_prob, x0 + bar_width, y1], fill=(50, 100, 200))
        # Draw softmax (red)
        draw.rectangle([x0 + bar_width, y0_soft, x0 + 2*bar_width, y1], fill=(200, 50, 50))

        # Optionally add numeric values above bars
        draw.text((x0, y0_prob - 18), f"{p:.2f}", fill="black", font=font)
        draw.text((x0 + bar_width, y0_soft - 18), f"{s:.2f}", fill="black", font=font)

        # Label categories on first row
        if i == 0:
            draw.text((x0 + 5, y1 + 2), labels[j], fill="black", font=font)

# Save the final diagram
out_path = OUT_DIR / "softmax_diagram.png"
canvas.save(out_path)
print(f"Saved softmax diagram to {out_path.resolve()}")


Saved softmax diagram to C:\Users\Jason Eckert\Documents\cv\02_ml_models\outputs\softmax_diagram.png
